# Options for encoding ordinal response

- Naive categorical encoding (ignore order)
- Integer encoding (really regression not classification)
- [Ordinal crossentropy loss for Keras](https://github.com/JHart96/keras_ordinal_categorical_crossentropy)
- [Ordinal regression for TF](https://github.com/gspell/TF-OrdinalRegression)
- "Cumulative" encoding [Cheng et al.]
- Split into K-1 binary classification problems [Frank and Hall] 
     - not sure if it's efficient with neural nets
     - Cheng et al.'s encoding does this in some sense, within a single network

In [1]:
# set up
# if installed, keras uses tf as backend
import keras
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Import ordinal crossentropy loss function
import sys
sys.path.insert(0, "./keras_ordinal_categorical_crossentropy")
import ordinal_categorical_crossentropy as OCC

Using TensorFlow backend.


Example from Keras docs
https://keras.io/getting-started/sequential-model-guide/#examples

In [2]:
# Generate dummy data
np.random.seed(123)
x_train = np.random.random((1000, 20))
y_int_train = np.random.randint(10, size=(1000, 1))
y_train = keras.utils.to_categorical(y_int_train, num_classes=10)
x_test = np.random.random((100, 20))
y_int_test = np.random.randint(10, size=(100, 1))
y_test = keras.utils.to_categorical(y_int_test, num_classes=10)

# define classification model
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# set SGD as optimizer
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [3]:
# naive multi-class classification
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 762us/step - loss: 2.4149 - accuracy: 0.1010
Epoch 2/20
1000/1000 [==============================] - 0s 53us/step - loss: 2.3687 - accuracy: 0.1030
Epoch 3/20
1000/1000 [==============================] - 0s 50us/step - loss: 2.3393 - accuracy: 0.1040
Epoch 4/20
1000/1000 [==============================] - 0s 51us/step - loss: 2.3187 - accuracy: 0.1180
Epoch 5/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3116 - accuracy: 0.1230
Epoch 6/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3159 - accuracy: 0.0990
Epoch 7/20
1000/1000 [==============================] - 0s 48us/step - loss: 2.3176 - accuracy: 0.0990
Epoch 8/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3134 - accuracy: 0.1030
Epoch 9/20
1000/1000 [==============================] - 0s 52us/step - loss: 2.3104 - accuracy: 0.1010
Epoch 10/20
1000/1000 [==============================] - 0s 63us/step - 

[2.2891061305999756, 0.11999999731779099]

In [4]:
# Classification with ordinal crossentropy loss
model.compile(loss=OCC.loss,
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 893us/step - loss: 3.0800 - accuracy: 0.1240
Epoch 2/20
1000/1000 [==============================] - 0s 53us/step - loss: 3.1003 - accuracy: 0.1140
Epoch 3/20
1000/1000 [==============================] - 0s 64us/step - loss: 3.1182 - accuracy: 0.1190
Epoch 4/20
1000/1000 [==============================] - 0s 72us/step - loss: 3.1640 - accuracy: 0.1070
Epoch 5/20
1000/1000 [==============================] - 0s 65us/step - loss: 3.1477 - accuracy: 0.1220
Epoch 6/20
1000/1000 [==============================] - 0s 67us/step - loss: 3.1741 - accuracy: 0.1250
Epoch 7/20
1000/1000 [==============================] - 0s 220us/step - loss: 3.2084 - accuracy: 0.1090
Epoch 8/20
1000/1000 [==============================] - 0s 52us/step - loss: 3.1725 - accuracy: 0.1170
Epoch 9/20
1000/1000 [==============================] - 0s 51us/step - loss: 3.1476 - accuracy: 0.1280
Epoch 10/20
1000/1000 [==============================] - 0s 52us/step -

[3.2259721755981445, 0.14000000059604645]

In [5]:
# Integer encoding
model_int = Sequential()
model_int.add(Dense(64, activation='relu', input_dim=20))
model_int.add(Dropout(0.5))
model_int.add(Dense(64, activation='relu'))
model_int.add(Dropout(0.5))
model_int.add(Dense(1, activation='relu'))

model_int.compile(
    ## or try sparse_categorical_crossentropy for integer targets
    loss='mean_squared_error',
    optimizer=sgd,
    metrics=['accuracy', 'mse'])

model_int.fit(x_train, y_int_train, epochs=20, batch_size=128)
score = model_int.evaluate(x_test, y_int_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 704us/step - loss: 15.0870 - accuracy: 0.0920 - mse: 15.0870
Epoch 2/20
1000/1000 [==============================] - 0s 53us/step - loss: 9.4177 - accuracy: 0.0930 - mse: 9.4177
Epoch 3/20
1000/1000 [==============================] - 0s 57us/step - loss: 9.2963 - accuracy: 0.0930 - mse: 9.2963
Epoch 4/20
1000/1000 [==============================] - 0s 62us/step - loss: 8.7801 - accuracy: 0.0950 - mse: 8.7801
Epoch 5/20
1000/1000 [==============================] - 0s 63us/step - loss: 8.6351 - accuracy: 0.0810 - mse: 8.6351
Epoch 6/20
1000/1000 [==============================] - 0s 68us/step - loss: 8.4900 - accuracy: 0.0840 - mse: 8.4900
Epoch 7/20
1000/1000 [==============================] - 0s 59us/step - loss: 8.4289 - accuracy: 0.0830 - mse: 8.4289
Epoch 8/20
1000/1000 [==============================] - 0s 59us/step - loss: 8.3415 - accuracy: 0.1000 - mse: 8.3415
Epoch 9/20
1000/1000 [==============================] - 0s 70

[9.062883377075195, 0.09000000357627869, 9.062883377075195]

In [6]:
# let's try it again but with sparse cross-entropy loss
# Integer encoding
model_sparse = Sequential()
model_sparse.add(Dense(64, activation='relu', input_dim=20))
model_sparse.add(Dropout(0.5))
model_sparse.add(Dense(64, activation='relu'))
model_sparse.add(Dropout(0.5))
model_sparse.add(Dense(10, activation='relu'))

model_sparse.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

model_sparse.fit(x_train, y_int_train, epochs=20, batch_size=128)
score = model_sparse.evaluate(x_test, y_int_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 899us/step - loss: 8.2245 - accuracy: 0.0970
Epoch 2/20
1000/1000 [==============================] - 0s 52us/step - loss: 4.2371 - accuracy: 0.0830
Epoch 3/20
1000/1000 [==============================] - 0s 54us/step - loss: 2.5750 - accuracy: 0.1070
Epoch 4/20
1000/1000 [==============================] - 0s 52us/step - loss: 2.3952 - accuracy: 0.1110
Epoch 5/20
1000/1000 [==============================] - 0s 51us/step - loss: 2.3369 - accuracy: 0.1040
Epoch 6/20
1000/1000 [==============================] - 0s 61us/step - loss: 2.3033 - accuracy: 0.1000
Epoch 7/20
1000/1000 [==============================] - 0s 51us/step - loss: 2.3200 - accuracy: 0.1010
Epoch 8/20
1000/1000 [==============================] - 0s 52us/step - loss: 2.3066 - accuracy: 0.1000
Epoch 9/20
1000/1000 [==============================] - 0s 52us/step - loss: 2.3191 - accuracy: 0.1060
Epoch 10/20
1000/1000 [==============================] - 0s 51us/step - 

[2.3025853633880615, 0.14000000059604645]

Remarks:

- Accuracy is pretty low across all three, but (sparse) integer encoding seems to work well
- Could be that architecture (ie, layers) and optimizer need tuning
- Oh yeah also because features are uncorrelated with response (duh!)

[TODO] Last three options...

### Takeaways

- Sparse categorical cross-entropy performs as well as categorical cross-entropy
- Ordinal cross-entropy performs worse than categorical cross-entropy!
- Integer encoding performs the worst

## Simulated ordinal response

Let's generate toy data for which we actually know the 
data generating process (DGP). This will provide a better
benchmark for the different approaches.

In [104]:
# 1. set parameters
K = 3 # response categories
N = 10000 # number of examples
P = 3 # number of features
# thresholds:
mu0 = 0
mu1 = 3.14
# set DGP parameters
b0 = 1
b1 = 2
b2 = -2
b3 = 1

In [105]:
# 2. generate features
# TODO: auto-generate x_mean and x_cov
x_mean = (1, 2, 0.5) 
x_cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
X = np.random.multivariate_normal(x_mean, x_cov, N) # dim: NxK
# X.shape

In [106]:
# 3. generate normal error
u = np.random.normal(size=(N,1)) # dim: Nx1
# u.shape

In [107]:
# 4. generate latent response
B = np.array([[b1], [b2], [b3]]) # dim: Kx1
y_latent = b0 + X.dot(B) + u # dim: Nx1
# y_latent.shape

In [108]:
# 5. generate ordinal response
y = np.digitize(y_latent, [mu0, mu1], right=1)
# y.shape

In [110]:
# create train / test split
from sklearn import model_selection
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,y,test_size = 0.1, random_state = 0)

# encode response as categorical
train_y_cat = keras.utils.to_categorical(train_y, num_classes=K)
test_y_cat = keras.utils.to_categorical(test_y, num_classes=K)

In [113]:
# cross-entropy loss
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=P))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(K, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(train_x, train_y_cat, epochs=20, batch_size=128)
score = model.evaluate(test_x, test_y_cat, batch_size=128)
score

Epoch 1/20
9000/9000 [==============================] - 1s 118us/step - loss: 0.8180 - accuracy: 0.6058
Epoch 2/20
9000/9000 [==============================] - 0s 42us/step - loss: 0.5611 - accuracy: 0.7544
Epoch 3/20
9000/9000 [==============================] - 0s 45us/step - loss: 0.4807 - accuracy: 0.8017
Epoch 4/20
9000/9000 [==============================] - 0s 47us/step - loss: 0.4489 - accuracy: 0.8090
Epoch 5/20
9000/9000 [==============================] - 1s 61us/step - loss: 0.4370 - accuracy: 0.8079
Epoch 6/20
9000/9000 [==============================] - 0s 42us/step - loss: 0.4258 - accuracy: 0.8173
Epoch 7/20
9000/9000 [==============================] - 0s 47us/step - loss: 0.4098 - accuracy: 0.8198
Epoch 8/20
9000/9000 [==============================] - 0s 43us/step - loss: 0.4076 - accuracy: 0.8232
Epoch 9/20
9000/9000 [==============================] - 1s 58us/step - loss: 0.4050 - accuracy: 0.8221
Epoch 10/20
9000/9000 [==============================] - 0s 42us/step - 

[0.3165572345256805, 0.8679999709129333]

In [114]:
# ordinal crossentropy loss
model.compile(loss=OCC.loss,
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(train_x, train_y_cat, epochs=20, batch_size=128)
score = model.evaluate(test_x, test_y_cat, batch_size=128)
score

Epoch 1/20
9000/9000 [==============================] - 1s 135us/step - loss: 0.5016 - accuracy: 0.8273
Epoch 2/20
9000/9000 [==============================] - 1s 62us/step - loss: 0.4937 - accuracy: 0.8311
Epoch 3/20
9000/9000 [==============================] - 0s 46us/step - loss: 0.4909 - accuracy: 0.8352
Epoch 4/20
9000/9000 [==============================] - 0s 45us/step - loss: 0.4907 - accuracy: 0.8309
Epoch 5/20
9000/9000 [==============================] - 0s 45us/step - loss: 0.4885 - accuracy: 0.8318
Epoch 6/20
9000/9000 [==============================] - 1s 62us/step - loss: 0.4882 - accuracy: 0.8352
Epoch 7/20
9000/9000 [==============================] - 0s 45us/step - loss: 0.4934 - accuracy: 0.8347
Epoch 8/20
9000/9000 [==============================] - 0s 45us/step - loss: 0.4856 - accuracy: 0.8348
Epoch 9/20
9000/9000 [==============================] - 0s 47us/step - loss: 0.4944 - accuracy: 0.8318
Epoch 10/20
9000/9000 [==============================] - 1s 62us/step - 

[0.41264120388031006, 0.8679999709129333]

In [115]:
# integer encoding
model_int = Sequential()
model_int.add(Dense(64, activation='relu', input_dim=P))
model_int.add(Dropout(0.5))
model_int.add(Dense(64, activation='relu'))
model_int.add(Dropout(0.5))
model_int.add(Dense(1, activation='relu'))

model_int.compile(
    ## or try sparse_categorical_crossentropy for integer targets
    loss='mean_squared_error',
    optimizer=sgd,
    metrics=['accuracy', 'mse'])

model_int.fit(train_x, train_y, epochs=20, batch_size=128)
score = model_int.evaluate(test_x, test_y, batch_size=128)
score

Epoch 1/20
9000/9000 [==============================] - 1s 111us/step - loss: 0.3652 - accuracy: 0.6689 - mse: 0.3652
Epoch 2/20
9000/9000 [==============================] - 1s 58us/step - loss: 0.1869 - accuracy: 0.7739 - mse: 0.1869
Epoch 3/20
9000/9000 [==============================] - 0s 42us/step - loss: 0.1685 - accuracy: 0.7849 - mse: 0.1685
Epoch 4/20
9000/9000 [==============================] - 0s 51us/step - loss: 0.1566 - accuracy: 0.7936 - mse: 0.1566
Epoch 5/20
9000/9000 [==============================] - 0s 49us/step - loss: 0.1475 - accuracy: 0.8021 - mse: 0.1475
Epoch 6/20
9000/9000 [==============================] - 1s 64us/step - loss: 0.1469 - accuracy: 0.8042 - mse: 0.1469
Epoch 7/20
9000/9000 [==============================] - 0s 43us/step - loss: 0.1427 - accuracy: 0.8098 - mse: 0.1427
Epoch 8/20
9000/9000 [==============================] - 0s 46us/step - loss: 0.1430 - accuracy: 0.8068 - mse: 0.1430
Epoch 9/20
9000/9000 [==============================] - 0s 43us

[0.11300325906276702, 0.843999981880188, 0.11300323903560638]

In [116]:
# sparse encoding
model_sparse = Sequential()
model_sparse.add(Dense(64, activation='relu', input_dim=P))
model_sparse.add(Dropout(0.5))
model_sparse.add(Dense(64, activation='relu'))
model_sparse.add(Dropout(0.5))
model_sparse.add(Dense(K, activation='relu'))

model_sparse.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

model_sparse.fit(train_x, train_y, epochs=20, batch_size=128)
score = model_sparse.evaluate(test_x, test_y, batch_size=128)
score

Epoch 1/20
9000/9000 [==============================] - 1s 116us/step - loss: 2.2852 - accuracy: 0.5666
Epoch 2/20
9000/9000 [==============================] - 0s 47us/step - loss: 2.0912 - accuracy: 0.5727
Epoch 3/20
9000/9000 [==============================] - 1s 62us/step - loss: 1.8268 - accuracy: 0.5726
Epoch 4/20
9000/9000 [==============================] - 0s 46us/step - loss: 2.9750 - accuracy: 0.5726
Epoch 5/20
9000/9000 [==============================] - 0s 45us/step - loss: 4.8086 - accuracy: 0.5726
Epoch 6/20
9000/9000 [==============================] - 0s 47us/step - loss: 4.8172 - accuracy: 0.5726
Epoch 7/20
9000/9000 [==============================] - 1s 63us/step - loss: 4.8882 - accuracy: 0.5726
Epoch 8/20
9000/9000 [==============================] - 0s 49us/step - loss: 5.0779 - accuracy: 0.5724
Epoch 9/20
9000/9000 [==============================] - 0s 46us/step - loss: 5.0345 - accuracy: 0.5726
Epoch 10/20
9000/9000 [==============================] - 0s 46us/step - 

[1.098612057685852, 0.5649999976158142]

## Observations

- Sparse integer encoding is actually the worst!
- Accuracy about the same for cross_entropy and ordinal_cross_entropy loss functions + categorical encoding

This is still using 